<a href="https://colab.research.google.com/github/tdiffendal/USAT/blob/master/Copy_of_census_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2020 Census Response Rate Analysis

### Theresa Diffendal, USA Today data intern, 06/2020

#### 2020 response rates from: https://2020census.gov/en/response-rates.html
#### 2010 response rates from: https://api.census.gov/data/2010/dec/responserate/variables.html
#### Demographic information in 2014-2018 ACS 5-year-estimate from: https://data2.nhgis.org/main

## Initial Load and Merge

In [185]:
import pandas as pd
import numpy as np

# in excel changed original dates from m/d/yyyy to yyyy-mm-dd
initial_df = pd.read_csv('https://raw.githubusercontent.com/tdiffendal/USAT/master/census-responses/decennialrr2020_working.csv')
# had to resave as UTF-8 CSV, hence the 2
crosswalk = pd.read_csv('https://raw.githubusercontent.com/tdiffendal/USAT/master/census-responses/decennialrr2020_crosswalkfile2.csv')
# states paired with region as defined by census map at https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf
regions = pd.read_csv('https://raw.githubusercontent.com/tdiffendal/USAT/master/census-responses/state_region.csv')

# merge responses and crosswalk
merged1 = pd.merge(initial_df, crosswalk, on='GEO_ID')

#merge merged1 with region data
merged = pd.merge(merged1, regions, on='State')

# create column showing responses not from internet
merged['not_int'] = merged.CRRALL - merged.CRRINT

#reorder columns to move State, Geo_Name and Geo_Type to front; also going to drop some values
cols = merged.columns.tolist()
cols = ['GEO_ID', 'RESP_DATE', 'State', 'Geo_Name', 'Region', 'Geo_Type', 
        'CRRINT', 'not_int', 'CRRALL']
merged = merged[cols]
merged = merged.rename(columns={'CRRINT':'internet', 'CRRALL':'2020_rate'})
merged

## Column Names

GEO_ID = Geographic Identifier

RESP_DATE = Posting Date

State = name of state (one of the 50 states, District of Columbia, Puerto Rico, or NaN)

Geo_Name = name of the tract, county, state

Region = region of the U.S. in which state is located as defined by census map at https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

Geo_Type = type of geography; possible answers include Census Tract, Congressional District, Consolidated City, Country, County, County Subdivision, Place, Region, State, Tribal Tract, Tribal Area

DRRINT = Daily Self-Response Rate - Internet

DRRALL = Daily Self-Response Rate – Overall

CRRINT = Cumulative Self-Response Rate - Internet; renamed internet

not_int = new calculated column showing response rate NOT from internet

CRRALL = Cumulative Self-Response Rate – Overall; renamed 2020_rate

DINTMIN = Minimum Daily Internet Self-Response Rate

DMIN = Minimum Daily Overall Self-Response Rate

CINTMIN = Minimum Cumulative Internet Self-Response Rate

CMIN = Minimum Cumulative Overall Self-Response Rate

DINTMAX = Maximum Daily Internet Self-Response Rate

DMAX = Maximum Daily Overall Self-Response Rate

CINTMAX = Maximum Cumulative Internet Self-Response Rate

CMAX = Maximum Cumulative Overall Self-Response Rate

DINTAVG = Average Daily Internet Self-Response Rate

DAVG = Average Daily Overall Self-Response Rate

CINTAVG = Average Cumulative Internet Self-Response Rate

CAVG = Average Cumulative Overall Self-Response Rate

DINTMED = Median Daily Internet Self-Response Rate

DMED = Median Daily Overall Self-Response Rate

CINTMED = Median Cumulative Internet Self-Response Rate

CMED = Median Cumulative Overall Self-Response Rate

## States

### Read in 2020 data

In [187]:
# create df with response rate by state
is_states = merged['Geo_Type'] == 'State'
states2020 = merged[is_states]
states2020 = states2020.rename(columns={"internet": "state_internet", "not_int" : "state_not_int", "2020_rate": "2020_state_rate"})

# print df and sort by highest cumulative response rate
states2020.sort_values(by='2020_state_rate', ascending=False)

,GEO_ID,RESP_DATE,State,Geo_Name,Region,Geo_Type,state_internet,state_not_int,2020_state_rate
55595,0400000US27,2020-06-17,Minnesota,Minnesota,Midwest,State,59.8,11.2,71.0
117774,0400000US55,2020-06-17,Wisconsin,Wisconsin,Midwest,State,56.4,12.1,68.5
50831,0400000US26,2020-06-17,Michigan,Michigan,Midwest,State,53.1,14.7,67.8
38079,0400000US19,2020-06-17,Iowa,Iowa,Midwest,State,53.3,14.5,67.8
64533,0400000US31,2020-06-17,Nebraska,Nebraska,Midwest,State,53.6,14.1,67.7
...,...,...,...,...,...,...,...,...,...
116938,0400000US54,2020-06-17,West Virginia,West Virginia,South,State,34.8,18.3,53.1
46322,0400000US23,2020-06-17,Maine,Maine,Northeast,State,38.7,14.2,52.9
70196,0400000US35,2020-06-17,New Mexico,New Mexico,West,State,38.8,11.2,50.0
1970,0400000US02,2020-06-17,Alaska,Alaska,West,State,38.2,8.8,47.0


### Read in 2010 states and join

In [188]:
# read in csvs with 2010 response data for states
states2010 = pd.read_csv('https://raw.githubusercontent.com/tdiffendal/USAT/master/census-responses/states2010.csv')

# merge with 2020 states
states = pd.merge(states2020, states2010, on='State')
#get the column names
cols = states.columns.tolist()
#only select columns we want
cols = ['GEO_ID', 'State', 'Region',
 '2020_state_rate', '2010_rate', '2000_rate']
states = states[cols]
states = states.rename(columns={'2000_rate':'2000_state_rate', '2010_rate':'2010_state_rate'})

#create column with difference in 2010 vs 2020 response rate
states['10_20_state_difference'] = states['2020_state_rate'] - states['2010_state_rate']

#print table sorted by 10-20 difference largest ---> smallest
states.sort_values(by='10_20_state_difference', ascending=False)

,GEO_ID,State,Region,2020_state_rate,2010_state_rate,2000_state_rate,10_20_state_difference
5,0400000US08,Colorado,West,64.7,72,75,-7.3
47,0400000US53,Washington,West,66.9,76,75,-9.1
44,0400000US49,Utah,West,65.8,75,74,-9.2
18,0400000US22,Louisiana,South,55.7,65,68,-9.3
27,0400000US31,Nebraska,Midwest,67.7,77,81,-9.3
...,...,...,...,...,...,...,...
9,0400000US12,Florida,South,58.7,74,71,-15.3
1,0400000US02,Alaska,West,47.0,64,67,-17.0
33,0400000US37,North Carolina,South,57.8,76,69,-18.2
40,0400000US45,South Carolina,South,56.0,75,68,-19.0


## Census Tracts

### 2020 Tracts

In [189]:
# select just census tract geo types
tracts2020 = merged[merged['Geo_Type'].str.contains("Tract")]
#rename column
tracts2020 = tracts2020.rename(columns={"2020_rate": "2020_tract_rate"})
# sort by highest cumulative response rate
tracts2020.sort_values(by='2020_tract_rate', ascending=False)

,GEO_ID,RESP_DATE,State,Geo_Name,Region,Geo_Type,internet,not_int,2020_tract_rate
26459,1400000US13215010606,2020-06-17,Georgia,"Tract 106.06, Muscogee",South,Census Tract,0.0,98.1,98.1
49682,1400000US25013812903,2020-06-17,Massachusetts,"Tract 8129.03, Hampden",Midwest,Census Tract,86.2,6.9,93.1
53285,1400000US26099223801,2020-06-17,Michigan,"Tract 2238.01, Macomb",Midwest,Census Tract,86.2,6.6,92.8
113068,1400000US51059492202,2020-06-17,Virginia,"Tract 4922.02, Fairfax",South,Census Tract,89.2,3.3,92.5
54097,1400000US26139021605,2020-06-17,Michigan,"Tract 216.05, Ottawa",Midwest,Census Tract,85.4,7.0,92.4
...,...,...,...,...,...,...,...,...,...
53490,1400000US26099982100,2020-06-17,Michigan,"Tract 9821, Macomb",Midwest,Census Tract,0.0,0.0,0.0
88882,1400000US40109980005,2020-06-17,Oklahoma,"Tract 9800.05, Oklahoma",South,Census Tract,0.0,0.0,0.0
88881,1400000US40109980003,2020-06-17,Oklahoma,"Tract 9800.03, Oklahoma",South,Census Tract,0.0,0.0,0.0
122998,2560000US2147T00100,2020-06-17,Tribal Land,"T001, Mashpee Wampanoag",Tribal Land,Tribal Tract,0.0,0.0,0.0


In [190]:
#tract rates compared to state averages
tractState2020 = pd.merge(tracts2020, states, on=['State', 'Region'])
tractState2020 = tractState2020[['GEO_ID_x', 'State', 'Geo_Name', 'Region','2020_tract_rate', '2020_state_rate', '2010_state_rate', '10_20_state_difference']]
tractState2020 = tractState2020.rename(columns={'GEO_ID_x':'GEO_ID'})
tractState2020['2020_tract_st_diff'] = tractState2020['2020_tract_rate'] - tractState2020['2020_state_rate']
tractState2020.sort_values(by=['2020_tract_st_diff'])
#merging with tracts with states will drop tribal tracts (as they have no state), so those are examined separately below

,GEO_ID,State,Geo_Name,Region,2020_tract_rate,2020_state_rate,2010_state_rate,10_20_state_difference,2020_tract_st_diff
38379,1400000US26083980100,Michigan,"Tract 9801, Keweenaw",Midwest,0.0,67.8,78,-10.2,-67.8
38744,1400000US26099982100,Michigan,"Tract 9821, Macomb",Midwest,0.0,67.8,78,-10.2,-67.8
40422,1400000US27007940001,Minnesota,"Tract 9400.01, Beltrami",Midwest,4.7,71.0,81,-10.0,-66.3
76532,1400000US49037942000,Utah,"Tract 9420, San Juan",West,0.8,65.8,75,-9.2,-65.0
76533,1400000US49037942100,Utah,"Tract 9421, San Juan",West,0.8,65.8,75,-9.2,-65.0
...,...,...,...,...,...,...,...,...,...
48992,1400000US35049010602,New Mexico,"Tract 106.02, Santa Fe",West,85.6,50.0,65,-15.0,35.6
71352,1400000US48121021535,Texas,"Tract 215.35, Denton",South,91.9,56.2,71,-14.8,35.7
48799,1400000US35028000500,New Mexico,"Tract 5, Los Alamos",West,87.0,50.0,65,-15.0,37.0
48570,1400000US35001003741,New Mexico,"Tract 37.41, Bernalillo",West,87.2,50.0,65,-15.0,37.2


### Read in 2010 tract rates and join

In [191]:
# read in csvs with 2010 response data for tracts and states
tracts2010 = pd.read_csv('https://raw.githubusercontent.com/tdiffendal/USAT/master/census-responses/2010responserate.csv')
#rename this column
tracts2010 = tracts2010.rename(columns={'FSRR2010':'2010_tract_rate'})
tracts2010

,NAME,county,State,GEO_ID,2010_tract_rate,state_num,county_num,tract
0,Census Tract 201,Autauga County,Alabama,1400000US01001020100,70.6,1,1,20100
1,Census Tract 202,Autauga County,Alabama,1400000US01001020200,70.1,1,1,20200
2,Census Tract 203,Autauga County,Alabama,1400000US01001020300,73.6,1,1,20300
3,Census Tract 204,Autauga County,Alabama,1400000US01001020400,78.4,1,1,20400
4,Census Tract 205.01,Autauga County,Alabama,1400000US01001020501,81.2,1,1,20501
...,...,...,...,...,...,...,...,...
84514,Census Tract 7505.01,Yauco Municipio,Puerto Rico,1400000US72153750501,70.4,72,153,750501
84515,Census Tract 7505.02,Yauco Municipio,Puerto Rico,1400000US72153750502,73.4,72,153,750502
84516,Census Tract 7505.03,Yauco Municipio,Puerto Rico,1400000US72153750503,63.4,72,153,750503
84517,Census Tract 7506.01,Yauco Municipio,Puerto Rico,1400000US72153750601,67.1,72,153,750601


In [192]:
## difference in row numbers: both tract dfs have 84519 rows, but when joined only 84093
# Identify what values are in tracts2010 and not in tracts2020
key_diff1 = set(tracts2010.GEO_ID).difference(tracts2020.GEO_ID)
len(key_diff1)
#key_diff1

# Identify what values are in tracts2020 and not in tracts2010
key_diff2 = set(tracts2020.GEO_ID).difference(tracts2010.GEO_ID)
len(key_diff2)
#key_diff2

# 2010 rates do not include tribal tracts while 2020 tracts are missing some 
# tracts in a multitude of states, likely due to a change in tract boundaries. 
# Those differences account for 426 tracts, which is .5% of the original 84519 
# tracts. As these tracts comprise a small percentage, they can be dropped. 

426

In [193]:
# merge with 2020 tracts
tracts = pd.merge(tractState2020, tracts2010, on='GEO_ID')
#get column names
cols = tracts.columns.tolist()
#select only columns we want
cols = ['Geo_Name','county', 'State_y', 'Region', '2020_tract_rate', '2010_tract_rate', '2020_tract_st_diff', '2020_state_rate', '2010_state_rate', '10_20_state_difference']
tracts = tracts[cols]
#rename weird column name
tracts = tracts.rename(columns={'State_y':'State'})
#print df sorted largest --> smallest 2010 rate
tracts.sort_values(by='2010_tract_rate', ascending=False)

,Geo_Name,county,State,Region,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,2020_state_rate,2010_state_rate,10_20_state_difference
10379,"Tract 109.06, San Bernardino",San Bernardino County,California,West,13.7,100.0,-48.8,62.5,73,-10.5
48849,"Tract 9400, Otero",Otero County,New Mexico,West,16.0,100.0,-34.0,50.0,65,-15.0
13467,"Tract 9800, Arapahoe",Arapahoe County,Colorado,West,87.5,100.0,22.8,64.7,72,-7.3
8287,"Tract 9800, Monterey",Monterey County,California,West,0.7,100.0,-61.8,62.5,73,-10.5
6449,"Tract 2653.01, Los Angeles",Los Angeles County,California,West,4.8,100.0,-57.7,62.5,73,-10.5
...,...,...,...,...,...,...,...,...,...,...
83055,"Tract 9402.02, Fremont",Fremont County,Wyoming,West,22.3,NaN,-32.6,54.9,69,-14.1
83056,"Tract 9403.01, Fremont",Fremont County,Wyoming,West,33.5,NaN,-21.4,54.9,69,-14.1
83057,"Tract 9403.02, Fremont",Fremont County,Wyoming,West,40.8,NaN,-14.1,54.9,69,-14.1
83058,"Tract 9404, Fremont",Fremont County,Wyoming,West,52.1,NaN,-2.8,54.9,69,-14.1


In [194]:
#how many null 2010 response values are there: 531, which is .6% of all rows, 84093
is_no_2010 = tracts.isnull()
no_2010 = is_no_2010.any(axis=1)
no_2010 = tracts[no_2010]
no_2010.sort_values(by="2010_tract_rate")

#due to the low percentage, these null values will be discarded
tracts = tracts.dropna(axis=0)
#check they'd discraded
tracts.sort_values(by='2010_tract_rate')
#there are no 2010 na values, so all were dropped

,Geo_Name,county,State,Region,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,2020_state_rate,2010_state_rate,10_20_state_difference
77029,"Tract 9663, Windsor",Windsor County,Vermont,Northeast,20.5,0.0,-34.0,54.5,69,-14.5
48861,"Tract 9408, Rio Arriba",Rio Arriba County,New Mexico,West,23.5,0.0,-26.5,50.0,65,-15.0
6111,"Tract 2074, Los Angeles",Los Angeles County,California,West,26.0,0.0,-36.5,62.5,73,-10.5
21943,"Tract 231.15, DeKalb",DeKalb County,Georgia,South,10.0,0.0,-47.4,57.4,72,-14.6
63439,"Tract 122, Centre",Centre County,Pennsylvania,Northeast,17.6,0.0,-47.2,64.8,77,-12.2
...,...,...,...,...,...,...,...,...,...,...
13467,"Tract 9800, Arapahoe",Arapahoe County,Colorado,West,87.5,100.0,22.8,64.7,72,-7.3
24218,"Tract 7.03, Ada",Ada County,Idaho,West,78.0,100.0,13.0,65.0,76,-11.0
73376,"Tract 202.07, Hidalgo",Hidalgo County,Texas,South,36.8,100.0,-19.4,56.2,71,-14.8
50116,"Tract 9704, Fulton",Fulton County,New York,Northeast,20.2,100.0,-36.5,56.7,69,-12.3


In [195]:
#create column with difference in 2010 vs 2020 response rate
tracts['10_20_tract_difference'] = tracts['2020_tract_rate'] - tracts['2010_tract_rate']
#sort df largest --> smallest 10-20 difference
tracts.sort_values(by='10_20_tract_difference', ascending=False)

,Geo_Name,county,State,Region,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,2020_state_rate,2010_state_rate,10_20_state_difference,10_20_tract_difference
23292,"Tract 106.06, Muscogee",Muscogee County,Georgia,South,98.1,0.0,40.7,57.4,72,-14.6,98.1
22281,"Tract 37, Fulton",Fulton County,Georgia,South,67.3,0.0,9.9,57.4,72,-14.6,67.3
65389,"Tract 364, Philadelphia",Philadelphia County,Pennsylvania,Northeast,80.2,13.5,15.4,64.8,77,-12.2,66.7
40288,"Tract 9824, Wayne",Wayne County,Michigan,Midwest,83.1,16.7,15.3,67.8,78,-10.2,66.4
21302,"Tract 107.07, Chatham",Chatham County,Georgia,South,68.2,2.4,10.8,57.4,72,-14.6,65.8
...,...,...,...,...,...,...,...,...,...,...,...
6449,"Tract 2653.01, Los Angeles",Los Angeles County,California,West,4.8,100.0,-57.7,62.5,73,-10.5,-95.2
53795,"Tract 1595.10, Suffolk",Suffolk County,New York,Northeast,2.9,100.0,-53.8,56.7,69,-12.3,-97.1
49059,"Tract 4.04, Albany",Albany County,New York,Northeast,1.8,100.0,-54.9,56.7,69,-12.3,-98.2
7980,"Tract 9800.21, Los Angeles",Los Angeles County,California,West,1.6,100.0,-60.9,62.5,73,-10.5,-98.4


## Demographic Data

In [196]:
#load both sets of demographic data, join
demo1 = pd.read_csv('https://raw.githubusercontent.com/tdiffendal/USAT/master/census-responses/demographics1_working.csv')
demo2 = pd.read_csv('https://raw.githubusercontent.com/tdiffendal/USAT/master/census-responses/demographics2_working.csv')
demo = pd.merge(demo1, demo2, on=['GISJOIN', "YEAR", "STATE", "STATEA", 
                                              'COUNTY', 'COUNTYA', 'TRACTA', 
                                              'Geo_Name', 'NAME_E'])

# merge to create new df with response rates and demos for all tracts
df1 = pd.merge(tracts, demo, on='Geo_Name')

#create new column adding up pop with rent > 30% income (homelessness marker)
df1['rent_30_more'] = (df1['rent_30_34.9'] + df1['rent_35_39.9'] 
+ df1['rent_40_49.9'] + df1['rent_50_over'])
# check to see if column created
pd.set_option('display.max_columns', 100)
df1

,Geo_Name,county,State,Region,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,2020_state_rate,2010_state_rate,10_20_state_difference,10_20_tract_difference,GISJOIN,YEAR,STATE,STATEA,COUNTY,COUNTYA,TRACTA,NAME_E,total_population,total_population_race,white_alone,black_alone,amerindian_alone,asian_alone,pacific_islander_alone,other_alone,two_or_more,two_more_including_other,two_more_excluding_other,total_education,no_school,some_school,diploma,ged,some_college,associate,bachelor,master,prof_school,doctorate,language_total,lang_english_only,lang_spanish,lang_spanish_limited_english,lang_other_indo_euro,lang_asian_pacific_island,lang_other,income_poverty_ratio,income_poverty_under_half,...,income_poverty_1.25_1.49,income_poverty_1.5_1.84,income_poverty_1.85_1.99,income_poverty_2_over,median_household_income,per_capita_income,employment_total,labor_force,civilian_labor_force,civilian_employed,civilian_unemployed,armed_forces,not_labor_force,total_houses,occupied_houses,vacant_houses,total_occupied_houses,owner_occupied,renter_occupied,median_gross_rent,rent_to_income,rent_less_10,rent_10_14.9,rent_15_19.9,rent_20_24.9,rent_25_29.9,rent_30_34.9,rent_35_39.9,rent_40_49.9,rent_50_over,rent_not_computed,total_computer_status,has_computer,dial_up_computer,broadband_computer,no_internet_computer,no_computer,us_pop,us_born,us_territory_born,us_born_abroad,us_naturalization,not_us_citizen,total_pr,pr_born,pr_us_born,pr_born_abroad,pr_naturalization,pr_not_us_citizen,rent_30_more
0,"Tract 201, Autauga",Autauga County,Alabama,South,64.4,70.6,5.1,59.3,72,-12.7,-6.2,G0100010020100,2014-2018,Alabama,1,Autauga County,1,20100,"Tract 201, Autauga Alabama",1923,1923,1609,161,0,17,0,0,136,0,136,1303,0,123,346,97,225,102,236,138,21,15,765,708,40,14,12,5,0,1923.0,41.0,...,52.0,59.0,12.0,1468.0,58625.0,31580.0,1562.0,966.0,966.0,931.0,35.0,0.0,596.0,779,765,14,765,570,195,827.0,195.0,21.0,23.0,0.0,5.0,0.0,4.0,14.0,46.0,45.0,37.0,765,625,0,549,76,140,1923,1870,0,11,18,24,0,0,0,0,0,0,109.0
1,"Tract 202, Autauga",Autauga County,Alabama,South,65.6,70.1,6.3,59.3,72,-12.7,-4.5,G0100010020200,2014-2018,Alabama,1,Autauga County,1,20200,"Tract 202, Autauga Alabama",2028,2028,819,1129,0,19,27,9,25,0,25,1443,11,234,499,87,283,87,182,48,5,7,719,699,0,0,16,4,0,1818.0,78.0,...,75.0,88.0,208.0,1016.0,43531.0,19376.0,1646.0,790.0,785.0,750.0,35.0,5.0,856.0,852,719,133,719,464,255,775.0,255.0,0.0,30.0,14.0,12.0,52.0,26.0,25.0,3.0,80.0,13.0,719,519,0,468,51,200,2028,1993,0,10,15,10,0,0,0,0,0,0,134.0
2,"Tract 203, Autauga",Autauga County,Alabama,South,73.8,73.6,14.5,59.3,72,-12.7,0.2,G0100010020300,2014-2018,Alabama,1,Autauga County,1,20300,"Tract 203, Autauga Alabama",3476,3476,2306,730,0,30,5,299,106,0,106,2406,26,281,708,142,628,166,249,187,0,19,1296,1196,48,0,15,37,0,3476.0,198.0,...,138.0,79.0,402.0,2170.0,51875.0,22527.0,2704.0,1684.0,1679.0,1624.0,55.0,5.0,1020.0,1397,1296,101,1296,841,455,917.0,455.0,0.0,5.0,50.0,43.0,73.0,35.0,30.0,23.0,135.0,61.0,1296,1081,0,950,131,215,3476,3231,5,60,49,131,0,0,0,0,0,0,223.0
3,"Tract 204, Autauga",Autauga County,Alabama,South,77.8,78.4,18.5,59.3,72,-12.7,-0.6,G0100010020400,2014-2018,Alabama,1,Autauga County,1,20400,"Tract 204, Autauga Alabama",3831,3831,3382,296,24,8,0,0,121,18,103,2769,14,151,744,175,508,272,602,177,74,52,1639,1578,31,16,13,9,8,3831.0,29.0,...,214.0,414.0,34.0,2909.0,54050.0,30527.0,3097.0,1752.0,1712.0,1663.0,49.0,40.0,1345.0,1867,1639,228,1639,1262,377,966.0,377.0,16.0,37.0,43.0,85.0,33.0,39.0,28.0,7.0,71.0,18.0,1639,1468,7,1315,146,171,3831,3720,0,30,30,51,0,0,0,0,0,0,145.0
4,"Tract 206, Autauga",Autauga County,Alabama,South,66.4,71.9,7.1,59.3,72,-12.7,-5.5,G0100010020600,2014-2018,Alabama,1,Autauga County,1,20600,"Tract 206, Autauga Alabama",3705,3705,2726,878,0,0,0,57,44,0,44,2455,43,367,678,194,507,161,346,105,22,32,1332,1307,16,0,9,0,0,3689.0,100.0,...,187.0,308.0,38.0,2030.0,46688.0,28049.0,3004.0,1451.0,1443.0,1333.0,110.0,8.0,1553.0,1413,1332,81,1332,1031,301,732.0,301.0,7.0,79.0,54.0,8.0,12.0,0

In [197]:
#df.std()Returns the standard deviation of each column
#df.corr()Returns the correlation between columns in a data frame

#In order to get # of null/missing values for each column, run 
pd.set_option('display.max_rows', 100)
pd.isnull(df1).sum()

# make a dataframe of all rows with na value
df2 = df1[df1.isna().any(axis=1)]
df2
# how many nas in each state
hm = df2['State'].value_counts()
hm = pd.DataFrame(hm)
hm = hm.reset_index().rename(columns={'index':'state', 'State':'count_na'})
hm
#compare to total 
hmm = df1['State'].value_counts()
hmm = pd.DataFrame(hmm)
hmm = hmm.reset_index().rename(columns={'index':'state', 'State':'count'})
hmm

#see what percent of state values of na
#Puerto Rico will lose the greatest % if these are dropped
hmmm = pd.merge(hm, hmm, on=['state'])
hmmm['na_percent'] = (hmmm['count_na']*100) / hmmm['count']
hmmm

#how many total nas? --> 1171
sum(hmmm['count_na'])

#nas account for what total % of all rows --> 1.82
(sum(hmmm['count_na'])*100) / sum(hmmm['count'])

#Less than 2% of total, so for now will drop those rows
# before df had 64413 rows
df = df1.dropna()    #now has 63242 rows, 1171 difference (total # nas)

In [198]:
#check if any nas
pd.set_option('display.max_rows', None)
print(pd.isnull(df).sum())

#still 52 states? (50 + DC and PR)
print("Number states: ", len(df['State'].unique()))

#compare tract numbers now to before na drop
df3 = pd.DataFrame(df1.groupby('State')['Geo_Name'].nunique()).rename(columns={'Geo_Name':'beforeDrop'})
df4 = pd.DataFrame(df.groupby('State')['Geo_Name'].nunique()).rename(columns={'Geo_Name':'afterDrop'})
df5 = pd.merge(df3, df4, left_index = True, right_index=True)
#number tracts dropped from each state
df5['numDrop'] = df5['beforeDrop'] - df5['afterDrop']
# the percentage of total tracts dropped
df5['dropPct'] = df5['numDrop']*100 / df5['beforeDrop']
df5 #puerto rico loses the most tracts at 5.7%

Geo_Name                        0
county                          0
State                           0
Region                          0
2020_tract_rate                 0
2010_tract_rate                 0
2020_tract_st_diff              0
2020_state_rate                 0
2010_state_rate                 0
10_20_state_difference          0
10_20_tract_difference          0
GISJOIN                         0
YEAR                            0
STATE                           0
STATEA                          0
COUNTY                          0
COUNTYA                         0
TRACTA                          0
NAME_E                          0
total_population                0
total_population_race           0
white_alone                     0
black_alone                     0
amerindian_alone                0
asian_alone                     0
pacific_islander_alone          0
other_alone                     0
two_or_more                     0
two_more_including_other        0
two_more_exclu

,beforeDrop,afterDrop,numDrop,dropPct
State,,,,
Alabama,954,939,15,1.572327
Alaska,122,121,1,0.819672
Arizona,1253,1232,21,1.675978
Arkansas,553,548,5,0.904159
California,6787,6719,68,1.001915
Colorado,1048,1025,23,2.194656
Connecticut,770,742,28,3.636364
Delaware,175,168,7,4.000000
District of Columbia,152,152,0,0.000000


In [199]:
#return column size
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_row', 50)

#print df with no nas
#df.to_csv("all_rates_demos_merged.csv")

## Analysis

In [200]:
#show dataframe with all years, states, tracts, demographics
df

#states has info on 2010 and 2020 states
#tracts has data on 2010 and 2020 tracts and states

,Geo_Name,county,State,Region,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,...,total_pr,pr_born,pr_us_born,pr_born_abroad,pr_naturalization,pr_not_us_citizen,rent_30_more
0,"Tract 201, Autauga",Autauga County,Alabama,South,64.4,70.6,5.1,...,0,0,0,0,0,0,109.0
1,"Tract 202, Autauga",Autauga County,Alabama,South,65.6,70.1,6.3,...,0,0,0,0,0,0,134.0
2,"Tract 203, Autauga",Autauga County,Alabama,South,73.8,73.6,14.5,...,0,0,0,0,0,0,223.0
3,"Tract 204, Autauga",Autauga County,Alabama,South,77.8,78.4,18.5,...,0,0,0,0,0,0,145.0
4,"Tract 206, Autauga",Autauga County,Alabama,South,66.4,71.9,7.1,...,0,0,0,0,0,0,101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64406,"Tract 7503, Yauco",Yauco Municipio,Puerto Rico,Puerto Rico,25.8,69.1,5.8,...,3960,3866,54,0,40,0,35.0
64407,"Tract 7504, Yauco",Yauco Municipio,Puerto Rico,Puerto Rico,13.7,52.0,-6.3,...,2685,2537,139,0,0,9,72.0
64408,"Tract 7505.01, Yauco",Yauco Municipio,Puerto Rico,Puerto Rico,30.6,70.4,10.6,...,6303,6148,135,0,0,20,94.0
64409,"Tract 7505.02, Yauco",Yauco Municipio,Puerto Rico,Puerto Rico,37.0,73.4,17.0,...,2316,2201,97,0,13,5,53.0


### National Comparative Rankings 2010 vs 2020

In [201]:
# Average difference between current state rates and 2010 state rates as of 6/15/20
#takes a while to run so commented out
#df.mean(axis=0)['10_20_state_difference']

#average 2020 response rate across states?
states.mean(axis=0)['2020_state_rate']

#assign ranks to states based on comparative response rate
states['2020_rank'] = states['2020_state_rate'].rank(method='max', ascending=False)
states['2010_rank'] = states['2010_state_rate'].rank(method='max', ascending=False)

#pull ranks into separate dataframe
state_ranks = states[['State', '2020_rank', '2010_rank']].sort_values(by='2020_rank')

#show change in rank from 2010 to 2020
#negative number means a state has a lower 2020 response rate and has gone down in rankings
state_ranks['rank_change'] = state_ranks['2010_rank'] - state_ranks['2020_rank']
state_ranks.sort_values(by='rank_change', ascending=True)

#see how many states only changed 2 or fewer positions
#small_change = state_ranks[state_ranks.rank_change.between(-2, 2, inclusive=True)].sort_values(by='rank_change')
#small_change
#16 states have stayed ~similar in the rankings, and this seems to impact
#states with both high and low response rates
#small_change.mean(axis=0)['2020_rank']

,State,2020_rank,2010_rank,rank_change
40,South Carolina,42.0,23.0,-19.0
33,North Carolina,34.0,20.0,-14.0
38,Pennsylvania,17.0,10.0,-7.0
42,Tennessee,26.0,20.0,-6.0
14,Indiana,10.0,4.0,-6.0
...,...,...,...,...
11,Hawaii,37.0,47.0,10.0
13,Illinois,9.0,20.0,11.0
44,Utah,11.0,23.0,12.0
47,Washington,6.0,20.0,14.0


In [202]:
df.groupby('State').mean().sort_values(by='2020_state_rate', ascending=False)
df.mean(axis=0)['2020_state_rate']

61.41236678157792

In [203]:
# average difference by region
states.groupby('Region').mean().sort_values(by='10_20_state_difference', ascending=False)

,2020_state_rate,2010_state_rate,2000_state_rate,10_20_state_difference,2020_rank,2010_rank
Region,,,,,,
Midwest,65.815385,77.307692,79.230769,-11.492308,11.461538,13.230769
West,59.369231,70.923077,72.538462,-11.553846,29.384615,35.846154
Northeast,59.687500,72.375000,72.875000,-12.687500,30.500000,31.250000
South,58.705882,72.176471,70.647059,-13.470588,32.647059,31.588235
Puerto Rico,20.000000,54.000000,54.000000,-34.000000,52.000000,52.000000


### 2010 Total Participation

In [204]:
states['2010_state_rate'].mean()   #72.83%
df['2010_state_rate'].mean()
#df['2010_tract_rate'].mean()  #67.79%

73.83452452484109

### State stats without Puerto Rico

In [205]:
no_pr = states[states.State != 'Puerto Rico']

# average non internet response rate
no_pr.mean(axis=0)['state_not_int']

KeyError: ignored

In [ ]:
# average internet response
no_pr.mean(axis=0)['state_internet']

In [ ]:
# average overall response rate
no_pr.mean(axis=0)['2020_state_rate']

In [ ]:
# average region response rate (so south now excludes Puerto Rico)
no_pr.groupby('Region').mean().sort_values(by='2020_state_rate', ascending=False)

### Internet Usage

In [ ]:
# highest non-internet response rate (not_int)
states.sort_values(by='state_not_int', ascending=False)

In [ ]:
states.mean(axis=0)['state_not_int']

In [ ]:
states.mean(axis=0)['state_internet']

In [ ]:
states.mean(axis=0)['2020_state_rate']

In [ ]:
# average by region
# NOTE as tribal tracts are not assigned to a state they do not have a corresponding region and thus are not counted in the regional calculations
states.groupby('Region').mean().sort_values(by='2020_state_rate', ascending=False)

In [ ]:
# average difference as of 6/15/20
#this can take a while to run so is commented out unless needed
#tracts.mean(axis=0)['10_20_tract_difference']

In [ ]:
# average difference by region
tracts.groupby('Region').mean().sort_values(by='10_20_tract_difference', ascending=False)

In [ ]:
#tract average differences vs state rates
tracts.groupby('State').mean().sort_values(by='2020_state_rate', ascending=False)

### Tribal tracts

In [ ]:
# create df with response rates in tribal tracts
tribal = tracts[tracts['Geo_Type'].str.contains("Tribal")]
tribal.sort_values(by='2020_tract_rate', ascending=False)

In [ ]:
### tribal areas and tracts stats

#mean non internet response
tribal.mean(axis=0)['not_int']

In [ ]:
# mean internet response rate
tribal.mean(axis=0)['internet']

In [ ]:
# mean overall response rate
tribal.mean(axis=0)['2020_tract_rate']

### Tracts with 0 overall response rate

In [ ]:
## Tracts with 0 cumulative response rate: 28
is_zero = tracts['2020_tract_rate'] == 0.0
zeros = tracts[is_zero]
zeros.sort_values(by='State')

## Tracts with 0 cumulative response rate: 28

#make dataframe of states with # tracts with 0%, number total tracts, and what % of total tracts are 0
temp = pd.DataFrame(zeros['State'].value_counts())
temp2 = pd.DataFrame(tracts['State'].value_counts())
temp3 = pd.merge(temp, temp2, right_index=True, left_index=True)
#rename the columns
temp3 = temp3.rename(columns={"State_x": "0_tracts", "State_y" : "total_tracts"})
#compute percentage
temp3['0_percent'] = temp3['0_tracts'] * 100 / temp3['total_tracts']
temp3

## Regressions

### With Puerto Rico

#### 2020 Regressions

In [ ]:
### 2020 Multi-regression

import statsmodels.api as sm

#put all variables for predicting 2020 rates in dataframe
variables20 = df[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target20 = df[["2020_tract_rate"]]

#build model and print summary
model20 = sm.OLS(target20, variables20).fit()
model20.summary()

In [ ]:
### 2020 linear regressions for each variable

from sklearn import linear_model

#create list of variable names
cols = variables20.columns.tolist()
#build linear model
regr = linear_model.LinearRegression()
#create empty list to store loop results
rows = []
#loop through each variable
for i in cols:
    #fit linear model to variable
    regr.fit(variables20[[i]], target20)
    #save model variable name, intercept, coef and r^2 to list
    rows.append([i, regr.intercept_, regr.coef_, regr.score(variables20[[i]], target20)])

#turn list into df with these column names
linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets lol
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['intercept'] = linears20['intercept'].str.get(0)

#print df sorted coefs largest --> smallest
linears20.sort_values(by='coefficient', ascending=False)

#### Normalized 2020 inputs

In [ ]:
### 2020 Multi regression with normalized variables

#normalize variable values
norm_variables20 = (variables20 - variables20.min()) / (variables20.max() - variables20.min())

#build normalized multi-regress model and print summary
norm_model20 = sm.OLS(target20, norm_variables20).fit()
norm_model20.summary()

In [ ]:
#2020 normalized linear regressions for each variable

# linear regression for each variable 'i'
cols = norm_variables20.columns.tolist()
# create model
norm_regr = linear_model.LinearRegression()
#empty list for loop results
rows = []
#loop through each variable
for i in cols:
    #fit model to each variable
    norm_regr.fit(norm_variables20[[i]], target20)
    #add model results to list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_, 
                 norm_regr.score(norm_variables20[[i]], target20)])

#turn list into dataframe
norm_linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['intercept'] = norm_linears20['intercept'].str.get(0)

#print df ordered by largest to smallest coef
norm_linears20.sort_values(by='coefficient', ascending=False)

#### 2010 Regressions

In [ ]:
### 2010 multi regression

#put all variables for predicting 2010 rates in dataframe
variables10 = df[['total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2010 response rates - in separate dataframe
target10 = df[["2010_tract_rate"]]

#create model and print summary table
model10 = sm.OLS(target10, variables10).fit()
model10.summary()

In [ ]:
### 2010 linear regression for each variable

#list of variable names
cols = variables10.columns.tolist()
#build multi-reg model
regr = linear_model.LinearRegression()
#create empty list for loop results
rows = []

#loop through variables
for i in cols:
    #fit a model to the current variable
    regr.fit(variables10[[i]], target10)
    #save the model's resulting variable name, intercept, coef, and r^2
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables10[[i]], target10)])

#turn list into data frame
linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['intercept'] = linears10['intercept'].str.get(0)

#print data frame ordered coefficient largest --> smallest
linears10.sort_values(by='coefficient', ascending=False)

#### Normalized 2010 Regressions

In [ ]:
### 2010 normalized multi-regression

#normalize the variables
norm_variables10 = (variables10 - variables10.min()) / (variables10.max() - variables10.min())

#build normalized model and print summary
norm_model10 = sm.OLS(target10, norm_variables10).fit()
norm_model10.summary()

In [ ]:
###2010 normalized linear regressions for each variable

#create list of variable names
cols = norm_variables10.columns.tolist()
#build the model
norm_regr = linear_model.LinearRegression()
#create empty list for model results
rows = []
#cycle through variables
for i in cols:
    #do the linear regression on the current variable
    norm_regr.fit(norm_variables10[[i]], target10)
    #add the corresponding variable name, intercept, coefficient and r-squared to the list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_,
                norm_regr.score(norm_variables10[[i]], target10)])

#turn list into data frame with these column names
norm_linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['intercept'] = norm_linears10['intercept'].str.get(0)

#print df sorted by coefficient
norm_linears10.sort_values(by='coefficient', ascending=False)

#### 2020 Edited Regressions
Edited = Run regressions with fewer variables

In [ ]:
### 2020 edited multi-regressions

#put all variables for predicting 2020 rates in dataframe
variables_ed = df[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other', 'income_poverty_ratio', 'per_capita_income',
 'civilian_employed', 'civilian_unemployed',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income','rent_30_more',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer',
 'us_born', 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target_ed = df[["2020_tract_rate"]]

#build and fit the multi-regression model
model_ed = sm.OLS(target_ed, variables_ed).fit()
#print out the model summary table
model_ed.summary()

In [ ]:
### 2020 edited linear regressions

#create list of variable names
cols = variables_ed.columns.tolist()
#build the model
regr = linear_model.LinearRegression()
#create empty list to append results
rows = []

#loop through variables
for i in cols:
    #fit the model
    regr.fit(variables_ed[[i]], target_ed)
    #put model variable name, intercept, coef and r^2 in list
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables_ed[[i]], target_ed)])

#turn list into data frame with these column names
linears_ed = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['intercept'] = linears_ed['intercept'].str.get(0)

#print df sorted by coefficient largest --> smallest
linears_ed.sort_values(by='coefficient', ascending=False)

In [ ]:
### normalized 2020 edited variables multi regression

#normalize the variables
norm_variables_ed = (variables_ed - variables_ed.min()) / (variables_ed.max() - variables_ed.min())

#build normalized model and print summary
norm_model_ed = sm.OLS(target_ed, norm_variables_ed).fit()
norm_model_ed.summary()

### Without Puerto Rico

#### 2020 regressions

In [ ]:
### 2020 Multi-regression

#df without puerto rico for regression 
no_pr = df[df['Region'] != 'Puerto Rico']
#this is different from earlier code no_pr = states[states.State != 'Puerto Rico']
#earlier code omitted Puerto Rico from state level data. this eliminates from tract level data


import statsmodels.api as sm

#put all variables for predicting 2020 rates in dataframe
variables20 = no_pr[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target20 = no_pr[["2020_tract_rate"]]

#build model and print summary
model20 = sm.OLS(target20, variables20).fit()
model20.summary()

In [ ]:
### 2020 linear regressions for each variable

from sklearn import linear_model

#create list of variable names
cols = variables20.columns.tolist()
#build linear model
regr = linear_model.LinearRegression()
#create empty list to store loop results
rows = []
#loop through each variable
for i in cols:
    #fit linear model to variable
    regr.fit(variables20[[i]], target20)
    #save model variable name, intercept, coef and r^2 to list
    rows.append([i, regr.intercept_, regr.coef_, regr.score(variables20[[i]], target20)])

#turn list into df with these column names
linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets lol
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['intercept'] = linears20['intercept'].str.get(0)

#print df sorted coefs largest --> smallest
linears20.sort_values(by='coefficient', ascending=False)

#### Normalized 2020 inputs

In [ ]:
### 2020 Multi regression with normalized variables

#normalize variable values
norm_variables20 = (variables20 - variables20.min()) / (variables20.max() - variables20.min())

#build normalized multi-regress model and print summary
norm_model20 = sm.OLS(target20, norm_variables20).fit()
norm_model20.summary()

In [ ]:
#2020 normalized linear regressions for each variable

# linear regression for each variable 'i'
cols = norm_variables20.columns.tolist()
# create model
norm_regr = linear_model.LinearRegression()
#empty list for loop results
rows = []
#loop through each variable
for i in cols:
    #fit model to each variable
    norm_regr.fit(norm_variables20[[i]], target20)
    #add model results to list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_, 
                 norm_regr.score(norm_variables20[[i]], target20)])

#turn list into dataframe
norm_linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['intercept'] = norm_linears20['intercept'].str.get(0)

#print df ordered by largest to smallest coef
norm_linears20.sort_values(by='coefficient', ascending=False)

#### 2010 Regressions

In [ ]:
### 2010 multi regression

#put all variables for predicting 2010 rates in dataframe
variables10 = no_pr[['total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2010 response rates - in separate dataframe
target10 = no_pr[["2010_tract_rate"]]

#create model and print summary table
model10 = sm.OLS(target10, variables10).fit()
model10.summary()

In [ ]:
### 2010 linear regression for each variable

#list of variable names
cols = variables10.columns.tolist()
#build multi-reg model
regr = linear_model.LinearRegression()
#create empty list for loop results
rows = []

#loop through variables
for i in cols:
    #fit a model to the current variable
    regr.fit(variables10[[i]], target10)
    #save the model's resulting variable name, intercept, coef, and r^2
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables10[[i]], target10)])

#turn list into data frame
linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['intercept'] = linears10['intercept'].str.get(0)

#print data frame ordered coefficient largest --> smallest
linears10.sort_values(by='coefficient', ascending=False)

#### Normalized 2010 Regressions

In [ ]:
### 2010 normalized multi-regression

#normalize the variables
norm_variables10 = (variables10 - variables10.min()) / (variables10.max() - variables10.min())

#build normalized model and print summary
norm_model10 = sm.OLS(target10, norm_variables10).fit()
norm_model10.summary()

In [ ]:
###2010 normalized linear regressions for each variable

#create list of variable names
cols = norm_variables10.columns.tolist()
#build the model
norm_regr = linear_model.LinearRegression()
#create empty list for model results
rows = []
#cycle through variables
for i in cols:
    #do the linear regression on the current variable
    norm_regr.fit(norm_variables10[[i]], target10)
    #add the corresponding variable name, intercept, coefficient and r-squared to the list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_,
                norm_regr.score(norm_variables10[[i]], target10)])

#turn list into data frame with these column names
norm_linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['intercept'] = norm_linears10['intercept'].str.get(0)

#print df sorted by coefficient
norm_linears10.sort_values(by='coefficient', ascending=False)

#### 2020 Edited Regressions
Edited = Run regressions with fewer variables

In [ ]:
### 2020 edited multi-regressions

#put all variables for predicting 2020 rates in dataframe
variables_ed = no_pr[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other', 'income_poverty_ratio', 'per_capita_income',
 'civilian_employed', 'civilian_unemployed',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income','rent_30_more',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer',
 'us_born', 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target_ed = no_pr[["2020_tract_rate"]]

#build and fit the multi-regression model
model_ed = sm.OLS(target_ed, variables_ed).fit()
#print out the model summary table
model_ed.summary()

In [ ]:
### 2020 edited linear regressions

#create list of variable names
cols = variables_ed.columns.tolist()
#build the model
regr = linear_model.LinearRegression()
#create empty list to append results
rows = []

#loop through variables
for i in cols:
    #fit the model
    regr.fit(variables_ed[[i]], target_ed)
    #put model variable name, intercept, coef and r^2 in list
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables_ed[[i]], target_ed)])

#turn list into data frame with these column names
linears_ed = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['intercept'] = linears_ed['intercept'].str.get(0)

#print df sorted by coefficient largest --> smallest
linears_ed.sort_values(by='coefficient', ascending=False)

In [ ]:
### normalized 2020 edited variables multi regression

#normalize the variables
norm_variables_ed = (variables_ed - variables_ed.min()) / (variables_ed.max() - variables_ed.min())

#build normalized model and print summary
norm_model_ed = sm.OLS(target_ed, norm_variables_ed).fit()
norm_model_ed.summary()

### Puerto Rico

In [ ]:
#create df with just puerto rico
is_pr = df['Region'] == 'Puerto Rico'
pr = df[is_pr]
pr

#### 2020 regressions

In [ ]:
### 2020 Multi-regression

import statsmodels.api as sm

#put all variables for predicting 2020 rates in dataframe
variables20 = pr[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target20 = pr[["2020_tract_rate"]]

#build model and print summary
model20 = sm.OLS(target20, variables20).fit()
model20.summary()

In [ ]:
### 2020 linear regressions for each variable

from sklearn import linear_model

#create list of variable names
cols = variables20.columns.tolist()
#build linear model
regr = linear_model.LinearRegression()
#create empty list to store loop results
rows = []
#loop through each variable
for i in cols:
    #fit linear model to variable
    regr.fit(variables20[[i]], target20)
    #save model variable name, intercept, coef and r^2 to list
    rows.append([i, regr.intercept_, regr.coef_, regr.score(variables20[[i]], target20)])

#turn list into df with these column names
linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets lol
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['intercept'] = linears20['intercept'].str.get(0)

#print df sorted coefs largest --> smallest
linears20.sort_values(by='coefficient', ascending=False)

#### Normalized 2020 inputs

In [ ]:
### 2020 Multi regression with normalized variables

#normalize variable values
norm_variables20 = (variables20 - variables20.min()) / (variables20.max() - variables20.min())

#build normalized multi-regress model and print summary
norm_model20 = sm.OLS(target20, norm_variables20).fit()
norm_model20.summary()

In [ ]:
#2020 normalized linear regressions for each variable

# linear regression for each variable 'i'
cols = norm_variables20.columns.tolist()
# create model
norm_regr = linear_model.LinearRegression()
#empty list for loop results
rows = []
#loop through each variable
for i in cols:
    #fit model to each variable
    norm_regr.fit(norm_variables20[[i]], target20)
    #add model results to list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_, 
                 norm_regr.score(norm_variables20[[i]], target20)])

#turn list into dataframe
norm_linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['intercept'] = norm_linears20['intercept'].str.get(0)

#print df ordered by largest to smallest coef
norm_linears20.sort_values(by='coefficient', ascending=False)

#### 2010 Regressions

In [ ]:
### 2010 multi regression

#put all variables for predicting 2010 rates in dataframe
variables10 = pr[['total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2010 response rates - in separate dataframe
target10 = pr[["2010_tract_rate"]]

#create model and print summary table
model10 = sm.OLS(target10, variables10).fit()
model10.summary()

In [ ]:
### 2010 linear regression for each variable

#list of variable names
cols = variables10.columns.tolist()
#build multi-reg model
regr = linear_model.LinearRegression()
#create empty list for loop results
rows = []

#loop through variables
for i in cols:
    #fit a model to the current variable
    regr.fit(variables10[[i]], target10)
    #save the model's resulting variable name, intercept, coef, and r^2
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables10[[i]], target10)])

#turn list into data frame
linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['intercept'] = linears10['intercept'].str.get(0)

#print data frame ordered coefficient largest --> smallest
linears10.sort_values(by='coefficient', ascending=False)

#### Normalized 2010 Regressions

In [ ]:
### 2010 normalized multi-regression

#normalize the variables
norm_variables10 = (variables10 - variables10.min()) / (variables10.max() - variables10.min())

#build normalized model and print summary
norm_model10 = sm.OLS(target10, norm_variables10).fit()
norm_model10.summary()

In [ ]:
###2010 normalized linear regressions for each variable

#create list of variable names
cols = norm_variables10.columns.tolist()
#build the model
norm_regr = linear_model.LinearRegression()
#create empty list for model results
rows = []
#cycle through variables
for i in cols:
    #do the linear regression on the current variable
    norm_regr.fit(norm_variables10[[i]], target10)
    #add the corresponding variable name, intercept, coefficient and r-squared to the list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_,
                norm_regr.score(norm_variables10[[i]], target10)])

#turn list into data frame with these column names
norm_linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['intercept'] = norm_linears10['intercept'].str.get(0)

#print df sorted by coefficient
norm_linears10.sort_values(by='coefficient', ascending=False)

#### 2020 Edited Regressions
Edited = Run regressions with fewer variables

In [ ]:
### 2020 edited multi-regressions

#put all variables for predicting 2020 rates in dataframe
variables_ed = pr[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other', 'income_poverty_ratio', 'per_capita_income',
 'civilian_employed', 'civilian_unemployed',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income','rent_30_more',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer',
 'us_born', 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target_ed = pr[["2020_tract_rate"]]

#build and fit the multi-regression model
model_ed = sm.OLS(target_ed, variables_ed).fit()
#print out the model summary table
model_ed.summary()

In [ ]:
### 2020 edited linear regressions

#create list of variable names
cols = variables_ed.columns.tolist()
#build the model
regr = linear_model.LinearRegression()
#create empty list to append results
rows = []

#loop through variables
for i in cols:
    #fit the model
    regr.fit(variables_ed[[i]], target_ed)
    #put model variable name, intercept, coef and r^2 in list
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables_ed[[i]], target_ed)])

#turn list into data frame with these column names
linears_ed = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['intercept'] = linears_ed['intercept'].str.get(0)

#print df sorted by coefficient largest --> smallest
linears_ed.sort_values(by='coefficient', ascending=False)

In [ ]:
### normalized 2020 edited variables multi regression

#normalize the variables
norm_variables_ed = (variables_ed - variables_ed.min()) / (variables_ed.max() - variables_ed.min())

#build normalized model and print summary
norm_model_ed = sm.OLS(target_ed, norm_variables_ed).fit()
norm_model_ed.summary()